# 02_entiry-extract

## read stored data from DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
dir_raw_id = 60
dir_process_id = 61

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='analysis_durians_kasikorn.json'))
meta

In [ ]:
data = json.load(fp)
data

In [ ]:
df = pd.json_normalize(data)
df

In [ ]:
df = df.drop([0])
df

In [ ]:
df[['date', 'month', 'year']] = df['publish_date'].str.split(' ', expand = True)
df = df.drop(columns=['publish_date'])
df

In [ ]:
df['month'] = df['month'].replace(['พฤษภาคม', 'กรกฎาคม'], ['05', '07'])
df['year'] = df['year'].astype(int) - 543
df['date'] = df['date'].astype(int).map("{:02}".format)
df

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df['year'] = df['year'].astype(str)

In [ ]:
df['publish_date'] = df['year'] + '-' + df['month'] + '-' + df['date']
df = df.drop(['date', 'month', 'year'], axis=1)
df

In [ ]:
df[['publish_date', 'retrieve_date']] = df[['publish_date', 'retrieve_date']].astype('datetime64')
df.info()

In [ ]:
df.columns

In [ ]:
df = df[['title', 'content', 'source_url', 'publish_date', 'retrieve_date']]
df

In [ ]:
df1 = df.copy()

## write DataNode to DataPlatform

In [ ]:
datanode.write(df=df1, 
               directory=dir_process_id, 
               name="The analysis durians from Kasikorn Research Center", 
               profiling=True, 
               lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='analysis_durians_kasikorn.json')])